In [43]:
import numpy as np
import pandas as pd

import scipy.stats as stats
from pandas.plotting import scatter_matrix

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.datasets import load_boston

In [3]:
def load_data():
    boston = load_boston()
    print (boston.DESCR)
    X = boston["data"]
    Y = boston["target"]
    names = boston["feature_names"]
    return X,Y,names

In [4]:
from sklearn.preprocessing import StandardScaler
def scale_data(X):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X

In [5]:
from sklearn.model_selection import train_test_split

def split_data(X,Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    return X_train, X_test, Y_train, Y_test

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def regression_predictor(model,x_test,y_test):
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    r2 = r2_score(y_test,y_pred)
    rmse = np.sqrt(mse)
    adj_r_squared = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)- x_test.shape[1]- 1)
    stats = pd.DataFrame({'rmse':rmse,'mse':mse,'mae':mae,'r2':r2,'adj_r_squared':adj_r_squared},index=['name'])
    return model,y_pred,stats

In [7]:
from sklearn.linear_model import LinearRegression
def linear_regression(X_train,y_train):
    regressor = LinearRegression()
    regressor.fit(X_train,y_train)
    return regressor

In [26]:
def plot_residuals(y_test,y_pred,name="Residual Plot"):
    residuals = y_test - y_pred
    plt.scatter(y_test,residuals)
    plt.title(name)
    plt.show()

In [48]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
def cross_validation_regressor(model,x_train,y_train):
    kf = KFold(n_splits=10, random_state=7)
    score = cross_val_score(model,x_train,y_train,cv=kf)
    return score.mean()

In [51]:
from sklearn.linear_model import Lasso
def lasso(x_train,x_test,y_train,y_test,alpha):
    lass = Lasso(alpha=alpha,random_state=7)
    model = lass.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    r2=model.score(x_test,y_test)
    rmse = np.sqrt(mse)
    val = cross_validation_regressor(lass,x_train,y_train)
    adj_r_squared = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)- x_test.shape[1]- 1)
    stats = pd.DataFrame({'cross_validation':val,
                        'rmse':rmse,'mse':mse,'mae':mae,'r2':(model.score(x_test,y_test)),'adj_r_squared':adj_r_squared},index=['name'])

    #stats = pd.DataFrame({'rmse':rmse,'mse':mse,'mae':mae,'r2':(model.score(x_test,y_test)),'adj_r_squared':adj_r_squared},index=['name'])
                                
    return model,y_pred,stats

In [52]:
from sklearn.linear_model import Ridge

def ridge(x_train,x_test,y_train,y_test,alpha):
    rid = Ridge(alpha=alpha,random_state=7,fit_intercept=True)
    model = rid.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    r2=model.score(x_test,y_test)
    rmse = np.sqrt(mse)
    val = cross_validation_regressor(rid,x_train,y_train)
    adj_r_squared = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)- x_test.shape[1]- 1)
    stats = pd.DataFrame({'cross_validation':val,
                        'rmse':rmse,'mse':mse,'mae':mae,'r2':(model.score(x_test,y_test)),'adj_r_squared':adj_r_squared},index=['name'])
    return model,y_pred,stats

In [29]:
from sklearn.linear_model import ElasticNet

def elasticnet(x_train,x_test,y_train,y_test,alpha):
    rid = ElasticNet(alpha=alpha,random_state=7,fit_intercept=True)
    model = rid.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    r2=model.score(x_test,y_test)
    rmse = np.sqrt(mse)
    #val = cross_validation_regressor(rid,x_train,y_train)
    adj_r_squared = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)- x_test.shape[1]- 1)
    stats = pd.DataFrame({'rmse':rmse,'mse':mse,'mae':mae,'r2':model.score(x_test,y_test),'adj_r_squared':adj_r_squared},index=['name'])
    return model,y_pred,stats

In [30]:
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def generate_regression_values(model, X, y):
    params = np.append(model.intercept_, model.coef_)
    predictions = model.predict(X)
    newX = pd.DataFrame({"Constant": np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((y - predictions) ** 2)) / (len(newX) - len(newX.columns))
    MSE = mean_squared_error(y,predictions)

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE * (np.linalg.inv(np.dot(newX.T, newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params / sd_b

    p_values = [2 * (1 - stats.t.cdf(np.abs(i), (len(newX) - 1))) for i in ts_b]

    sd_b = np.round(sd_b, 3)
    ts_b = np.round(ts_b, 3)
    p_values = np.round(p_values, 3)
    params = np.round(params, 4)

    myDF3 = pd.DataFrame()
    myDF3["Coefficients"], myDF3["Standard Errors"], myDF3["t values"], myDF3[
        "Probabilites"
    ] = [params,sd_b, ts_b, p_values]
    print(myDF3)

In [31]:
X,Y,names = load_data()

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [32]:

X = scale_data(X)

In [33]:
X_train, X_test, y_train, y_test = split_data(X,Y)

In [34]:
model = linear_regression(X_train,y_train)
#val = cross_validation_regressor(model,X_train,y_train)
model,y_pred,status = regression_predictor(model, X_test, y_test)
#print ("Linear model: ", pretty_print_linear(model.coef_, names, sort = True))

print("\n Model Used:\n", model)
print("\n Model Statistics:\n",status)

generate_regression_values(model, X_test, y_test)
plot_residuals(y_test,y_pred)



 Model Used:
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

 Model Statistics:
       adj_r_squared       mae        mse        r2      rmse
name       0.702558  3.151288  20.747143  0.725852  4.554903
    Coefficients  Standard Errors  t values  Probabilites
0        22.4647            0.367    61.165         0.000
1        -1.0998            0.412    -2.669         0.008
2         0.8806            0.512     1.721         0.087
3         0.4017            0.820     0.490         0.625
4         0.8221            0.430     1.912         0.058
5        -1.8779            0.743    -2.528         0.012
6         2.7330            0.545     5.010         0.000
7        -0.3596            0.675    -0.533         0.595
8        -2.9940            0.738    -4.059         0.000
9         2.0399            0.981     2.078         0.039
10       -1.3811            1.120    -1.233         0.219
11       -2.0113            0.548    -3.671         0.000
12        1

In [53]:
model,y_pred,status = lasso(X_train,X_test,y_train,y_test,alpha=0.3)

print("\n Model Used:\n", model)
print("\n Model Statistics:\n",status)

generate_regression_values(model, X_test, y_test)
plot_residuals(y_test,y_pred)


 Model Used:
 Lasso(alpha=0.3, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=7,
   selection='cyclic', tol=0.0001, warm_start=False)

 Model Statistics:
       adj_r_squared  cross_validation       mae       mse        r2      rmse
name       0.663402          0.683638  3.302667  23.47833  0.689762  4.845444
    Coefficients  Standard Errors  t values  Probabilites
0        22.5148            0.391    57.626         0.000
1        -0.2896            0.438    -0.661         0.510
2         0.0000            0.544     0.000         1.000
3        -0.0000            0.872    -0.000         1.000
4         0.7363            0.457     1.610         0.109
5        -0.4249            0.790    -0.538         0.591
6         2.8562            0.580     4.922         0.000
7        -0.0000            0.718    -0.000         1.000
8        -0.9108            0.785    -1.161         0.247
9         0.0000            1.044     0.

In [54]:
model,y_pred,status = ridge(X_train,X_test,y_train,y_test,alpha=0.3)

print("\n Model Used:\n", model)
print("\n Model Statistics:\n",status)

generate_regression_values(model, X_test, y_test)
plot_residuals(y_test,y_pred)


 Model Used:
 Ridge(alpha=0.3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=7, solver='auto', tol=0.001)

 Model Statistics:
       adj_r_squared  cross_validation       mae        mse        r2      rmse
name       0.702442          0.692328  3.150524  20.755217  0.725745  4.555789
    Coefficients  Standard Errors  t values  Probabilites
0        22.4650            0.367    61.154         0.000
1        -1.0949            0.412    -2.657         0.009
2         0.8736            0.512     1.707         0.090
3         0.3927            0.820     0.479         0.633
4         0.8239            0.430     1.916         0.057
5        -1.8665            0.743    -2.512         0.013
6         2.7353            0.546     5.013         0.000
7        -0.3605            0.675    -0.534         0.594
8        -2.9812            0.738    -4.041         0.000
9         2.0139            0.982     2.052         0.042
10       -1.3587            1.120    -1.2

In [39]:
model,y_pred,status = elasticnet(X_train,X_test,y_train,y_test,alpha=0.3)

print("\n Model Used:\n", model)
print("\n Model Statistics:\n",status)

generate_regression_values(model, X_test, y_test)



 Model Used:
 ElasticNet(alpha=0.3, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=7, selection='cyclic', tol=0.0001, warm_start=False)

 Model Statistics:
       adj_r_squared       mae        mse        r2      rmse
name       0.668218  3.260832  23.142398  0.694201  4.810655
    Coefficients  Standard Errors  t values  Probabilites
0        22.5284            0.388    58.077         0.000
1        -0.5055            0.435    -1.162         0.247
2         0.1951            0.541     0.361         0.719
3        -0.0376            0.866    -0.043         0.965
4         0.8515            0.454     1.875         0.063
5        -0.7029            0.785    -0.896         0.372
6         2.7987            0.576     4.858         0.000
7        -0.1623            0.713    -0.228         0.820
8        -1.1416            0.779    -1.466         0.145
9         0.0000            1.037     0.000      